In [38]:
import spacy
import re
import string
from neo4j.v1 import GraphDatabase, basic_auth

In [39]:
nlp = spacy.load('en')
driver = GraphDatabase.driver("bolt://localhost:7687", auth = basic_auth("neo4j", "temppassword1"))
session = driver.session()

In [55]:
f = open('dart-database/test.txt', 'r')

In [56]:
for line in f:
    spaceSplit = re.split('\s+', line)
    sentSplit = re.split('\"', line)
    source = sentSplit[3]
    dest = sentSplit[1]
    sentence = sentSplit[5]
    freq = spaceSplit[0]
    
    sourceDoc = nlp(string.capwords(source))
    sourceType = sourceDoc[0].ent_type_
    destDoc = nlp(string.capwords(dest))
    destType = destDoc[0].ent_type_
    
    sourceClass = source
    destClass = dest
    
    if source == "person/place/organization" or source == "person/place/organisation":
        session.run("MERGE (a:PPO {class: \"PPO\"})")
        sourceClass = "PPO"
    elif source == "person":
        session.run("MERGE (a:PER {class: \"PER\"})")
        sourceClass = "PER"
    elif source == "place":
        session.run("MERGE (a:PLC {class: \"PLC\"})")
        sourceClass = "PLC"
    elif source == "organization" or source == "organisation":
        session.run("MERGE (a:ORGS {class: \"ORGS\"})")
        sourceClass = "ORGS"
    else:
        if sourceType != "":
            session.run("MERGE (a: NamedEntity {class: {class}, type: {type}})", {"class": source, "type": sourceType})
        else:
            session.run("MERGE (a: Entity {class: {src}})", {"src": source})
    
    if dest == "person/place/organization" or dest == "person/place/organisation":
        session.run("MERGE (b:PPO {class: \"PPO\"})")
        destClass = "PPO"
    elif dest == "person":
        session.run("MERGE (b:PER {class: \"PER\"})")
        destClass = "PER"
    elif dest == "place":
        session.run("MERGE (b:PLC {class: \"PLC\"})")
        destClass = "PLC"
    elif dest == "organization" or dest == "organisation":
        session.run("MERGE (b:ORGS {class: \"ORGS\"})")
        destClass = "ORGS"
    else:
        if destType != "":
            session.run("MERGE (b: NamedEntity {class: {class}, type: {type}})", {"class": dest, "type": destType})
        else:
            session.run("MERGE (b: Entity {class: {src}})", {"src": dest})
    
    if sourceType != "" and destType != "":
        session.run("MATCH (a), (b)"
                   +"WHERE a.class = {sourceClass} AND a.type = {sourceType}"
                   +"AND b.class = {destClass} AND b.type = {destType}"
                   +"MERGE (a)-[r:CAN_BE {frequency: {freq}}]->(b)",
                   {"sourceClass": sourceClass, "sourceType": sourceType, "destClass": destClass, "destType": destType, "freq": freq})
    elif sourceType != "":
        session.run("MATCH (a), (b)"
                   +"WHERE a.class = {sourceClass} AND a.type  = {sourceType}"
                   +"AND b.class = {destClass}"
                   +"MERGE (a)-[r:CAN_BE {frequency: {freq}}]->(b)",
                   {"sourceClass": sourceClass, "sourceType": sourceType, "destClass": destClass, "freq": freq})
    elif destType != "":
        session.run("MATCH (a), (b)"
                   +"WHERE a.class = {sourceClass}"
                   +"AND b.class = {destClass} AND b.type  = {destType}"
                   +"MERGE (a)-[r:CAN_BE {frequency: {freq}}]->(b)",
                   {"sourceClass": sourceClass, "destClass": destClass, "destType": destType, "freq": freq})
    else:
        session.run("MATCH (a), (b)"
                   +"WHERE a.class = {sourceClass} AND b.class = {destClass}"
                   +"MERGE (a)-[r:CAN_BE {frequency: {freq}}]->(b)", 
                    {"sourceClass": sourceClass, "destClass": destClass, "freq": freq})